<a href="https://colab.research.google.com/github/niranjana-2004/ML-Project-6/blob/main/MovieRecommendationFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

df = pd.read_csv(
    "tmdb_movies_data.csv",
    engine="python",
    on_bad_lines="skip"
)

df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000.0,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562.0,6.5,2015.0,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000.0,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185.0,7.1,2015.0,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000.0,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480.0,6.3,2015.0,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000.0,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292.0,7.5,2015.0,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000.0,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947.0,7.3,2015.0,174799923.1,1.385749e+09


In [5]:
df = df[['original_title', 'overview', 'genres', 'keywords']]
df.head()

,original_title,overview,genres,keywords
0,Jurassic World,Twenty-two years after the events of Jurassic ...,Action|Adventure|Science Fiction|Thriller,monster|dna|tyrannosaurus rex|velociraptor|island
1,Mad Max: Fury Road,An apocalyptic story set in the furthest reach...,Action|Adventure|Science Fiction|Thriller,future|chase|post-apocalyptic|dystopia|australia
2,Insurgent,Beatrice Prior must confront her inner demons ...,Adventure|Science Fiction|Thriller,based on novel|revolution|dystopia|sequel|dyst...
3,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,Action|Adventure|Science Fiction|Fantasy,android|spaceship|jedi|space opera|3d
4,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,Action|Crime|Thriller,car race|speed|revenge|suspense|car


In [7]:
df['content'] = (
    df['overview'] + " " +
    df['genres'] + " " +
    df['keywords']
)

/tmp/ipython-input-1208345002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = (


In [6]:
df.fillna("",inplace=True)

/tmp/ipython-input-849151069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna("",inplace=True)


In [17]:
df = df.drop_duplicates(subset='original_title').reset_index(drop=True)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000
)

tfidf_matrix = tfidf.fit_transform(df['content'])

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf_matrix)

In [20]:
def recommend(movie_name, n=5):
    if movie_name not in df['original_title'].values:
        return "Movie not found in dataset."
    idx = df[df['original_title'] == movie_name].index[0]
    scores = list(enumerate(similarity[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    movie_indices = [i[0] for i in scores[1:n+1]]
    return df['original_title'].iloc[movie_indices]

In [24]:
#recommend("Avatar")
#recommend("The Dark Knight")
recommend("Titanic")
#recommend("Inception")
#df.columns

,original_title
7277,Death Ship
2294,Ghost Ship
10434,Ship of Fools
3813,Captain Phillips
4844,The Poseidon Adventure


In [25]:
import pickle
pickle.dump(df, open("movies.pkl", "wb"))
pickle.dump(similarity, open("similarity.pkl", "wb"))